In [1]:
from tokenizers import decoders, models, normalizers, pre_tokenizers, processors, trainers, Tokenizer
from collections import defaultdict
import torch
import pickle
from re import sub
import torch
import torch.optim as optim
import os
import matplotlib.pyplot as plt
import numpy as np
import sys
from tqdm import tqdm
import torch.nn as nn
import time
# from custom_transformer import CustomTransformer
import torch.nn.functional as F
import random
import wandb

wandb.init(project="transformers")



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: basujindal. Use `wandb login --relogin` to force relogin


In [2]:
# class TextLoader:
#     def __init__(self, PATH) -> None:
        
#         self.corpus = open(PATH, 'r').read()
#         self.sentences = self.corpus.split('\n')

#     def remove_rare_words(self,vocab_dict, min_repeat):

#         wordList = [k for k,v in vocab_dict.items() if v >= min_repeat]
#         vocab = set(wordList)
#         print(len(vocab), len(wordList))

    
#     def get_all_chars(self):
#         chars = defaultdict(lambda: 0)
#         for sen in self.sentences:
#             for char in sen:
#                 chars[char] += 1

#         return chars

#     def remove_sentence_with_rare_chars(self, min_count, dict):
#         sens_idx = []
#         for idx, sen in enumerate(self.sentences):
#             flag = 1
#             for char in sen:
#                 if(dict[char] < min_count):
#                     flag = 0
#                     break
#             if flag:
#                 sens_idx.append(idx)
#         return sens_idx

#     def create_cleaned_data(self, idxs, ext):
#         cleaned_sentences = [self.sentences[i] for i in idxs]
#         self.sentences = cleaned_sentences
#         print(len(self.sentences))

#         clean_file = open('cleaned_train.' + ext, 'w+')
#         for i in range(len(self.sentences)):  
#             clean_file.write(self.sentences[i] + '\n')
            
#     def batch_iterator(self, batch_size):
#         for i in range(0, len(self.sentences), batch_size):
#             yield self.sentences[i : i + batch_size]


In [3]:
# loader_en = TextLoader('datasets/train.en')
# loader_de = TextLoader('datasets/train.de')

In [4]:
# chars_en = loader_en.get_all_chars()
# chars_de = loader_de.get_all_chars()

# chars = defaultdict(lambda:0)
# for k,v in chars_en.items():
#     chars[k] += v
# for k,v in chars_de.items():
#     chars[k] += v

# chars = dict(sorted(chars.items(), key=lambda item: item[1], reverse=True))
# # dict(sorted(chars_de.items(), key=lambda item: item[1], reverse=True))

# cleaned_en = loader_en.remove_sentence_with_rare_chars(1000, chars)
# cleaned_de = loader_de.remove_sentence_with_rare_chars(1000, chars)
# cleaned_idx = list(set(cleaned_en) & set(cleaned_de))
# len(cleaned_idx), len(cleaned_en), len(cleaned_de)

# loader_de.create_cleaned_data(cleaned_idx, 'de')
# loader_en.create_cleaned_data(cleaned_idx, 'en')


In [5]:
# loader_en = TextLoader('datasets/cleaned_train.en')
# loader_de = TextLoader('datasets/cleaned_train.de')

# assert(len(loader_de.sentences) == len(loader_en.sentences))

In [6]:
# tokenizer_en = Tokenizer(models.BPE())
# tokenizer_en.normalizer = normalizers.Lowercase()
# # tokenizer_en.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# tokenizer_en.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

# trainer_en = trainers.BpeTrainer(vocab_size=25000,special_tokens=["[PAD]"])
# tokenizer_en.train_from_iterator(loader_en.batch_iterator(1000), trainer=trainer_en)

# # tokenizer_en.decoder =  decoders.ByteLevel()
# tokenizer_en.save("tokenizer_en_25000_start_token_SOS.json")

In [7]:
# tokenizer_en.decode([527, 1263, 197, 627, 48, 0, 0, 0])

In [8]:
# encoding = tokenizer_en.encode("My name is Basu")
# print(encoding.tokens,encoding.ids )
# tokenizer_en.decode(encoding.ids)

# for batch in loader_en.batch_iterator(2):
#     encoding = tokenizer_en.encode_batch(batch)
#     print(tokenizer_en.decode_batch([encoding[i].ids for i in range(len(encoding))]))
#     break

In [9]:
# li = []
# for batch in loader_en.batch_iterator(1024):
#     encoding = tokenizer_en.encode_batch(batch)
#     for i in range(len(encoding)):
#         li.append(len(encoding[i].ids))

# li = sorted(li, reverse=True)

In [10]:
# for i in range(5000, 6000):
#     if li[i] <= 100:
#         print(i)

# import matplotlib.pyplot as plt
# plt.hist(li)

In [11]:
# tokenizer_de = Tokenizer(models.BPE())
# tokenizer_de.normalizer = normalizers.Lowercase()
# # tokenizer_de.pre_tokenizer = pre_tokenizers.ByteLevel(add_prefix_space=False)

# tokenizer_de.pre_tokenizer = pre_tokenizers.WhitespaceSplit()

# # print(tokenizer.pre_tokenizer.pre_tokenize_str("Ich liebe das wirklich ."))

# trainer_de = trainers.BpeTrainer(vocab_size=25000, special_tokens=["[PAD]", "[SOS]", "[EOS]"])
# tokenizer_de.train_from_iterator(loader_de.batch_iterator(1000), trainer=trainer_de)

# tokenizer_de.post_processor = processors.TemplateProcessing(
#     single=f"[SOS]:0 $A:0 [EOS]:0",
#     special_tokens=[("[SOS]", 0), ("[EOS]", 1)])
# # tokenizer_de.decoder =  decoders.ByteLevel()

# tokenizer_de.save("tokenizer_de_25000_start_token_SOS.json")

In [12]:
# tokenizer_de.decode([i for i in range(300)])

In [13]:
# encoding = tokenizer_de.encode("My name is Basu")
# print(encoding.tokens, encoding.ids)
# tokenizer_de.decode(encoding.ids + [0,0,0,0])

In [14]:
# for batch in loader_de.batch_iterator(1024):
#     encoding = tokenizer_de.encode_batch(batch)
#     a = tokenizer_de.decode_batch([encoding[i].ids for i in range(len(encoding))])

In [15]:
# en_encodes = []
# encodes_len = []
# for idx, batch in enumerate(loader_en.batch_iterator(1024)):
#     encoding = tokenizer_en.encode_batch(batch)
#     for i in range(len(encoding)):
#         en_encodes.append(encoding[i].ids)
#         encodes_len.append(len(encoding[i].ids))

# de_encodes = []
# for idx, batch in enumerate(loader_de.batch_iterator(1024)):
#     encoding = tokenizer_de.encode_batch(batch)
#     for i in range(len(encoding)):
#         de_encodes.append(encoding[i].ids)

# encodes_len , en_encodes, de_encodes = (list(t) for t in zip(*sorted(zip(encodes_len , en_encodes, de_encodes))))


# # Dump the tokenized lists as pickle files for faster retrieval
# with open('datasets/en_tokenized.pkl', 'wb') as f:
#     pickle.dump(en_encodes, f)

# with open('datasets/de_tokenized.pkl', 'wb') as f:
#     pickle.dump(de_encodes, f)

In [16]:
with open('datasets/en_tokenized.pkl', 'rb') as f:
    en_tokens = pickle.load(f)

with open('datasets/de_tokenized.pkl', 'rb') as f:
    de_tokens = pickle.load(f)

print(len(en_tokens))
assert(len(de_tokens) == len(en_tokens))

4442492


In [17]:
def remove_large_Sentences(en_tokens, de_tokens, max_size):
    li = []
    for i in range(len(en_tokens)):
        if(len(en_tokens[i]) <= max_size and len(de_tokens[i]) <= max_size):
            li.append(i)

    return li

In [18]:
tokenizer_de = Tokenizer.from_file("tokenizer_de_25000_start_token_SOS.json")
tokenizer_en = Tokenizer.from_file("tokenizer_en_25000_start_token_SOS.json")

In [19]:
class MultiHeadAttention(nn.Module):
    def __init__(self, h, edim):
        super().__init__()

        self.h = h
        self.edim = edim
        self.dk = self.edim//self.h
        self.key = nn.Linear(self.edim,self.edim)
        self.query = nn.Linear(self.edim,self.edim)
        self.value = nn.Linear(self.edim,self.edim)
        self.linear = nn.Linear(self.edim,self.edim)
        

    def forward(self, key, value,query, mask = None):

        bs = key.shape[0]
        nwords_key = key.shape[1]
        nwords_query = query.shape[1]

        k = self.key(key).reshape(bs, nwords_key, self.h, self.dk).transpose(1,2)
        q = self.query(query).reshape(bs, nwords_query, self.h, self.dk).transpose(1,2)
        v = self.value(value).reshape(bs, nwords_key, self.h, self.dk).transpose(1,2)
        x = torch.einsum('bhmd,bhnd -> bhmn',(q,k))
        
        if mask != None:
            x = x.masked_fill(mask == False, float("-1e20"))

        x = F.softmax(x/(self.dk)**0.5, dim=3)

        x = torch.einsum('bhmn,bhnv -> bhmv', (x,v))
        x = x.transpose(1,2)

        x = x.reshape(bs, nwords_query, -1)
        x = self.linear(x)
        return x
    

class EncoderBlock(nn.Module):
    def __init__(self, edim, h, hdim, dropout):
        super().__init__()

        self.multiHeadAttention = MultiHeadAttention(h, edim)
        self.norm1 = nn.LayerNorm(edim)
        self.norm2 = nn.LayerNorm(edim)
        self.fc1 = nn.Linear(edim, hdim)
        self.fc2 = nn.Linear(hdim, edim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)


    def forward(self, src_embed, src_mask):

        x = self.multiHeadAttention(src_embed,src_embed,src_embed, src_mask)
        x = self.dropout1(x)
        subLayer1 = self.norm1(x + src_embed)

        x = self.fc2(self.relu(self.fc1(subLayer1)))
        x = self.dropout2(x)
        subLayer2 = self.norm2(x + subLayer1)

        return subLayer2

class DecoderBlock(nn.Module):
    def __init__(self,edim, h, hdim, dropout):
        super().__init__()

        self.multiHeadAttention = MultiHeadAttention(h, edim)
        self.maskedMultiHeadAttention = MultiHeadAttention(h, edim)
        self.norm1 = nn.LayerNorm(edim)
        self.norm2 = nn.LayerNorm(edim)
        self.norm3 = nn.LayerNorm(edim)
        self.fc1 = nn.Linear(edim, hdim)
        self.fc2 = nn.Linear(hdim, edim)
        self.relu = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)
        self.dropout3 = nn.Dropout(dropout)

    def forward(self, tgt_embed, src_encoded, src_mask, tgt_mask):

        x = self.maskedMultiHeadAttention(tgt_embed, tgt_embed, tgt_embed, tgt_mask)
        x = self.dropout1(x)
        subLayer1 = self.norm1(x + tgt_embed)

        x = self.multiHeadAttention(src_encoded, src_encoded, subLayer1, src_mask)
        x = self.dropout2(x)
        subLayer2 = self.norm2(x + subLayer1)

        x = self.fc2(self.relu(self.fc1(subLayer2)))
        x = self.dropout3(x)
        subLayer3 = self.norm3(x + subLayer2)
        
        return subLayer3


class Encoder(nn.Module):
    def __init__(self, nx, edim, h, hdim,dropout):
        super().__init__()

        self.nx = nx
        self.transformers = nn.ModuleList([EncoderBlock(edim, h, hdim,dropout) for _ in range(nx)])

    def forward(self, src_embed, src_mask):
        for block in self.transformers:
            embed = block(src_embed, src_mask)
        return embed

class Decoder(nn.Module):
    def __init__(self, nx, edim, h, hdim,dropout):
        super().__init__()

        self.nx = nx
        self.transformers = nn.ModuleList([DecoderBlock(edim, h, hdim,dropout) for _ in range(nx)])

    def forward(self, encoded, tgt_embed, src_mask, tgt_mask):

        for block in self.transformers:
            embed = block(tgt_embed, encoded, src_mask, tgt_mask)
        return embed


class CustomTransformer(nn.Module):
    def __init__(self, nx, edim, h, hdim, dropout, src_vocab_size, tgt_vocab_size):
        super().__init__()
        self.tgt_vocab_size = tgt_vocab_size
        self.src_embedding = nn.Embedding(src_vocab_size,edim)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size,edim)
        self.encoder = Encoder(nx, edim, h, hdim,dropout)
        self.decoder = Decoder(nx, edim, h, hdim,dropout)
        self.fc = nn.Linear(edim, tgt_vocab_size)
        self.dropout1 = nn.Dropout(dropout)
        self.dropout2 = nn.Dropout(dropout)

    def forward(self, src_tokens, tgt_tokens, src_mask, tgt_mask, src_pos_embed, tgt_pos_embed):

        src_embed = self.src_embedding(src_tokens) + src_pos_embed
        src_embed = self.dropout1(src_embed)
        tgt_embed = self.tgt_embedding(tgt_tokens) + tgt_pos_embed
        tgt_embed = self.dropout2(tgt_embed)
        encoded = self.encoder(src_embed, src_mask)
        decoded = self.decoder(encoded, tgt_embed, src_mask, tgt_mask)
        output = self.fc(decoded)

        return output

In [20]:
def loader(en_tokens, de_tokens, bs, pad_idx, shuffle = True):
    num_batches = len(en_tokens)//bs
    idxs = [i for i in range(num_batches)]

    if shuffle:
        random.shuffle(idxs)

    for i in idxs:
        max_len = len(en_tokens[(i+1)*bs - 1])
        en_tensor = torch.tensor([enc + 
        [pad_idx]*(max_len - len(enc)) for enc in en_tokens[i*bs:(i+1)*bs]])
        
        en_pad = torch.tensor((en_tensor != pad_idx)).unsqueeze(1).unsqueeze(2)

        de_lens = [len(enc) for enc in de_tokens[i*bs:(i+1)*bs]]
        max_len= max(de_lens)
        de_tensor = torch.tensor([enc + 
        [pad_idx]*(max_len - len(enc)) for enc in de_tokens[i*bs:(i+1)*bs]])
        

        de_mask = torch.ones(bs, max_len, max_len).tril().unsqueeze(1)


        yield en_tensor, en_pad, de_tensor, de_mask, de_lens


def positionEmbeding(edim, max_nwords):
    pos_emb = torch.zeros((max_nwords, edim))
    for pos in range(max_nwords):
        for i in range(edim//2):
            pos_emb[pos, 2*i] = np.sin(pos/(10000**(2*i/edim)))
            pos_emb[pos, 2*i + 1] = np.cos(pos/(10000**(2*i/edim)))

    return pos_emb

In [21]:
class NoamOpt:

    def __init__(self, model_size, warmup, optimizer):
        self.optimizer = optimizer
        self._step = 0
        self.warmup = warmup
        self.model_size = model_size
        self._rate = 0
        
    def step(self):
        "Update parameters and rate"
        self._step += 1
        rate = self.rate()
        for p in self.optimizer.param_groups:
            p['lr'] = rate
        self._rate = rate
        self.optimizer.step()
        
    def rate(self):
        return self.model_size ** (-0.5) *min(self._step ** (-0.5),  self._step * self.warmup ** (-1.5))

In [22]:
nx = 6
edim = 512
hdim = 2048
h = 8
src_vocab_size = 25000
tgt_vocab_size = 25000
n_epochs = 10
bs = 32
pad_idx = 0
dropout = 0.1
max_nwords = 100

max_idxs = remove_large_Sentences(en_tokens, de_tokens, max_nwords)
en_tokens = [en_tokens[i] for i in max_idxs]
de_tokens = [de_tokens[i] for i in max_idxs]
assert(len(de_tokens) == len(en_tokens))


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
net = CustomTransformer(nx, edim, h, hdim, dropout, src_vocab_size, tgt_vocab_size).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), betas=[0.9, 0.98])
scheduler = NoamOpt(edim,4000,optimizer)

wandb.config = {
  "nsteps": 500000,
  "batch_size": bs
}

cuda


In [23]:
n_epochs = 1
ep = 0

posEmb = positionEmbeding(edim, max_nwords)
for epoch in range(n_epochs):  # loop over the dataset multiple times

    correct, total = 0, 0
    p_bar=tqdm(total=len(en_tokens)//bs,desc="Percentage ")
  
    for src, src_mask, tgt, tgt_mask, tgt_lens in loader(en_tokens, de_tokens, bs, pad_idx):
        p_bar.update(1)


        labels = torch.cat([tgt[ii][1:tgt_lens[ii]] for ii in range(tgt.shape[0])], dim = 0)
        ep+=1
        src = src.to(device)
        src_mask = src_mask.to(device)
        tgt = tgt.to(device)
        tgt_mask = tgt_mask.to(device)


        src_pos_embed = posEmb[:src.shape[1]].unsqueeze(0).to(device)
        tgt_pos_embed = posEmb[:tgt.shape[1]].unsqueeze(0).to(device)

        optimizer.zero_grad()
        outputs = net(src, tgt, src_mask, tgt_mask, src_pos_embed, tgt_pos_embed)
        # print(outputs.shape[0])

        li = [outputs[ii][:tgt_lens[ii]-1] for ii in range(outputs.shape[0])]
        probs = torch.cat(li, dim = 0)


        # print(tokenizer_de.decode(li2[0].to('cpu').numpy()),
        #  tokenizer_de.decode(tgt[0].detach().to('cpu').numpy()))

        loss = criterion(probs, labels)
        # print(labels, probs, loss, labels.shape, probs.shape)       
        # loss.backward)
        optimizer.step()
        scheduler.step()
        wandb.log(
            {"loss": loss.data,
             "lr": scheduler._rate})

        # Optional
        wandb.watch(net)

        # break
        # total += labels.size(0)
        # correct += (predicted == labels).sum().item()

#         # print statistics
#         running_loss += loss.item()
#         if ep % num_steps == 0: 
#             val_accu = val(valloader)
#             print("epoch {0} | loss: {1:.2f} | Val Accuracy: {2:.2f} %".format(epoch, running_loss/num_steps, val_accu ))
#             running_loss = 0.0
#             if val_accu > best_accu:
#               best_accu= val_accu 
#               torch.save(net.state_dict(), 'net_val.pth')
#               print("Saving")
#             net.train()
#     print("Train accuracy: {0:.2f} %".format(100 * correct / total))

# print('Finished Training')

Percentage :   0%|          | 0/138394 [00:00<?, ?it/s]/tmp/ipykernel_54505/2227623443.py:13: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  en_pad = torch.tensor((en_tensor != pad_idx)).unsqueeze(1).unsqueeze(2)
Percentage :  11%|█         | 15358/138394 [18:43<2:08:46, 15.92it/s] 

KeyboardInterrupt: 

Percentage :  11%|█         | 15359/138394 [18:54<2:08:46, 15.92it/s]

In [ ]:
# from transformers import PreTrainedTokenizerFast

# fast_tokenizer = PreTrainedTokenizerFast(tokenizer_file="tokenizer_de_25000_start_token_SOS.json")

# for batch in loader_de.batch_iterator(1024):
#     # print(batch)
#     encoding = fast_tokenizer(batch)
#     # decoded = fast_tokenizer.batch_decode(encoding['input_ids'])
#     # print(decoded)
#     # break